## Exporting models for use on client-side apps

## Preprocessing models

My model was using transfer learning from Resnet-50. So, I will need to either
- create a model with both, or
- extract both models separately and use the predictions of one in the other

Architecturally, it makes more sense to go the later way, because then the last layer can be easily replaced while using this for other apps

#### Loading the base resnet model which will be used as a feeder

In [20]:
from keras.applications.resnet50 import ResNet50, preprocess_input

base_resnet_model = ResNet50(weights='imagenet', include_top=False)

In [21]:
## save the resnet model (for use in tensorflow-js; not required otherwise)

## TODO

#### Loading the saved weights for the last trained layer

In [22]:
from keras.models import load_model

last_layers_model = load_model('./keras_models/weights.extra_layers.hdf5')
last_layers_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 500)               1024500   
_________________________________________________________________
dropout_7 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 133)               66633     
Total params: 1,091,133.0
Trainable params: 1,091,133.0
Non-trainable params: 0.0
_________________________________________________________________


#### Testing the workflow

In [7]:
from glob import glob
dog_names = [item[20:-1] for item in sorted(glob("../dogImages/train/*/"))]
print('Number of dogs:', len(dog_names))

Number of dogs: 111


In [8]:
### function converts an image path into a tensor that can be input into the resnet model
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [9]:
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image    
import numpy as np

def predict_breed(img_path):
    tensor = path_to_tensor(img_path)
    bottleneck_feature = base_resnet_model.predict(preprocess_input(tensor))
    predicted_vector = last_layers_model.predict(bottleneck_feature)
    return dog_names[np.argmax(predicted_vector)]

In [10]:
predict_breed('../sample_images/image_4.jpg')

'34.Boxer'

## Android

Deploying to android requires a .pb format. Use the following function, to convert your model 

In [23]:
import os
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import import_pb_to_tensorboard

def keras_to_tensorflow(keras_model, 
        output_dir, 
        model_name,
        out_prefix="output_", 
        log_tensorboard=True):

    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], 
            out_prefix + str(i + 1))

    sess = K.get_session()


    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(
            sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, 
        name=model_name, as_text=False)

    if log_tensorboard:
        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name),
            output_dir)

AttributeError: module 'tensorflow.python.pywrap_tensorflow' has no attribute 'TFE_DEVICE_PLACEMENT_EXPLICIT'

In [12]:
## save both models
import os
import tensorflow as tf
from keras import backend as K

keras_to_tensorflow(base_resnet_model, 'android', 'base_resnet_model')
keras_to_tensorflow(last_layers_model, 'android', 'last_layers_model')

NameError: name 'keras_to_tensorflow' is not defined

## IOS Core ML

In [1]:
import coremltools

def saveCoreMLModel(model):
    coreml_model = coremltools.converters.keras.convert(model,
    input_names=['input'], output_names=['probs'],
        image_input_names='input',
        predicted_feature_name='dogBreed',
        class_labels='') ## fix
    coreml_model.save('resnet50custom.mlmodel')
    print('coreml saved')

ModuleNotFoundError: No module named 'coremltools'

In [12]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [1]:
from keras.models import load_model

model = load_model('weights.best.Resnet50.hdf5')
print(model)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


/home/akshata/.conda/envs/dog-project/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [9]:

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)


def predictBreed(img_path):
    predicted_vector = model.predict(path_to_tensor(img_path))
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

In [27]:
base_resnet_model = ResNet50(weights='imagenet', include_top=False)

/home/akshata/.local/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [28]:
base_resnet_model.predict(preprocess_input(path_to_tensor(img_path)))

NameError: name 'img_path' is not defined

In [ ]:
model.add(InputLayer(input_shape=(3*img_width*img_height)))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 500)               1024500   
_________________________________________________________________
dropout_7 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 133)               66633     
Total params: 1,091,133
Trainable params: 1,091,133
Non-trainable params: 0
_________________________________________________________________


In [10]:
from keras.preprocessing import image  
import numpy as np

In [30]:
def keras_to_tensorflow(keras_model, output_dir, 
    model_name,out_prefix="output_", 
        log_tensorboard=True):

    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], 
            out_prefix + str(i + 1))

    sess = K.get_session()

    from tensorflow.python.framework import graph_util
    from tensorflow.python.framework import graph_io

    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(
            sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, 
        name=model_name, as_text=False)

    if log_tensorboard:
        from tensorflow.python.tools import import_pb_to_tensorboard

        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name),
            output_dir)

In [26]:
import os
import tensorflow as tf
from keras import backend as K

keras_to_tensorflow(base_resnet_model, 'baseresnet', 'base_resnet_model')

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 318 variables.
INFO:tensorflow:Converted 318 variables to const ops.
Instructions for updating:
Use tf.gfile.GFile.
Model Imported. Visualize by running: tensorboard --logdir=baseresnet


INFO:tensorflow:Froze 318 variables.
INFO:tensorflow:Converted 318 variables to const ops.
Model Imported. Visualize by running: tensorboard --logdir=toplayers
